In [1]:
from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "charged-city-421819"
LOCATION   = "EU"
DATASET    = "covid"
TABLE      = "owid"

client = bigquery.Client(project=PROJECT_ID)

# 1) Ensure dataset (EU)
ds_id = f"{PROJECT_ID}.{DATASET}"
try:
    ds = client.get_dataset(ds_id)
    if ds.location != LOCATION:
        raise RuntimeError(f"Dataset exists in {ds.location}, expected {LOCATION}")
except Exception:
    ds = bigquery.Dataset(ds_id)
    ds.location = LOCATION
    client.create_dataset(ds, exists_ok=True)

# 2) Load OWID (chunked)
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
usecols = [
    "iso_code","location","date","population",
    "total_cases","new_cases","total_deaths","new_deaths",
    "people_vaccinated","people_fully_vaccinated","total_boosters"
]
first = True
for chunk in pd.read_csv(url, usecols=usecols, parse_dates=["date"], chunksize=200_000):
    chunk["date"] = chunk["date"].dt.date
    num = [c for c in chunk.columns if c not in ("iso_code","location","date")]
    chunk[num] = chunk[num].apply(pd.to_numeric, errors="coerce")
    job = client.load_table_from_dataframe(
        chunk, f"{ds_id}.{TABLE}",
        job_config=bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition="WRITE_TRUNCATE" if first else "WRITE_APPEND"
        )
    )
    job.result()
    first = False

print("Rows in table:", client.get_table(f"{ds_id}.{TABLE}").num_rows)

Rows in table: 429435
